In [1]:
import pandas as pd
import sys

from pycaret.classification import *
from sklearn.metrics import average_precision_score

split_df=pd.read_csv('annotations.csv')

In [2]:
split_df['lag_word']=split_df.Word.shift(1)
split_df['new_sentence']=np.where(split_df.lag_word.isin(['.','?','!']),1,0)

In [4]:
target='label'
cat_fea=['pos','stop_word','new_sentence']
num_fea=['synt']+[x for x in split_df.columns if 'word_2vec_' in x or 'bert_' in x]
use_df=split_df[[target]+cat_fea+num_fea]

# add important features of neighbour words, to aggregate the contextual information
# this will increase the f1 score from ~0.85 to ~0.89
lag_feas=['synt','pos','word_2vec_71','bert_158','bert_718','word_2vec_40','bert_306','stop_word']
for col in lag_feas:
    use_df['prev_%s'%col]=use_df[col].shift(1)
    use_df['next_%s'%col]=use_df[col].shift(-1)   

# pyycaret sometimes raise an error for LGB if we feed the raw data to it
# so it will be better if we one hot it in advance and rename them
use_df=pd.concat((pd.get_dummies(use_df[['prev_pos','next_pos']]),
                  use_df[cat_fea+num_fea+[target]]),axis=1)

org_names=use_df.columns.tolist()
use_df.columns=['col_%s'%x for x in range(len(org_names))]
target='col_%s'%org_names.index(target) # find the label in our columns

use_df.head()

col_0  col_1  col_2  col_3  col_4  col_5  col_6  col_7  col_8  col_9  \
0      0      0      0      0      0      0      0      0      0      0   
1      0      0      0      0      0      0      0      0      0      0   
2      0      0      0      0      0      0      0      0      0      1   
3      0      0      0      0      0      0      0      0      0      0   
4      0      0      0      0      0      0      0      0      0      0   

   col_10  col_11  col_12  col_13  col_14  col_15  col_16  col_17  col_18  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       1       0       0       0       0       0   
4       0       0       0       0       0       0       0       1       0   

   col_19  col_20  col_21  col_22  col_23  col_24  col_25  col_26  col_27  \
0       0       0       0       0       1       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   col_28  col_29  col_30  col_31  col_32  col_33  col_34  col_35  col_36  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       1       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   col_37  col_38  col_39  col_40  col_41  col_42  col_43  col_44  col_45  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   col_46  col_47  col_48  col_49  col_50  col_51  col_52  col_53  col_54  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       1       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   col_55  col_56  col_57  col_58  col_59  col_60  col_61  col_62  col_63  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       1       0       0       0       0       0       0       0       0   

   col_64  col_65  col_66  col_67  col_68  col_69  col_70  col_71  col_72  \
0       0       0       0       0       0       0       0       0       0   
1       0       1       0       0       0       0       0       0       0   
2       0       0       0       0       0       0       0       0       0   
3       0       0       0       0       0       0       0       0       0   
4       0       0       0       0       0       0       0       0       0   

   col_73  col_74  col_75  col_76  col_77  col_78  col_79  col_80  col_81  \
0       0       0       0       0       0       0       0       0       0   
1       0       0       0       0       0       0       0       0       0   
2       0       0       1       0       0       0       0       0       0   
3  

In [5]:
%%time
clf1 = setup(data = use_df,target = target,fold=10)
add_metric('apc', 'APC', average_precision_score, target = 'pred_proba') # add APC for this imbalanced data
best = compare_models(sort = 'f1',exclude = ['svm','knn','qda','nb'],n_select = 9) # return top 9 best models

Wall time: 2h 46min 1s


In [6]:
# return test f1 
for model in best:
    pred_holdout = predict_model(model)

In [10]:
plot_model(best[0], plot = 'feature')

In [11]:
# get the most important features' name
cols=[128,102,101,59,120,97,54,47,104,51]
res=[]
for col in cols:
    res.append(['col_%s'%col,org_names[col]])
    
pd.DataFrame(res,columns=['column_name','original_name'])

column_name original_name
0     col_128          synt
1     col_102  next_pos_NNP
2     col_101   next_pos_NN
3      col_59   prev_pos_NN
4     col_120  next_pos_VBP
5      col_97   next_pos_JJ
6      col_54   prev_pos_IN
7      col_47    prev_pos_.
8     col_104  next_pos_NNS
9      col_51   prev_pos_DT